In [49]:
import pandas as pd
import numpy as np
import sklearn as sk
from datetime import timedelta, date
from sodapy import Socrata


In [ ]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.seattle.gov", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.seattle.gov',
                 'sTbG6jfU0iYQQ463GDals2WXx',
                timeout=120)

#data is delayed 48 hrs
# socrata data keys for parking data
#2019 ytd
data_ytd = 'qktt-2bsy'
#last 30 days
data_mtd = 'rke9-rsvs'
# last 48 hours
data_48hrs = 'hiyf-7edq'


# Number of records to read at a time
loop_size = df_len = 1000
# list of dataframes
dfs = []

# columns that are numeric
num_cols = ['paidoccupancy','parkingspacecount','parkingtimelimitcategory','sourceelementkey']

i = 0
#check to see if last record was full or not
while loop_size == df_len:
    #print(i)
    print('records read: %.3fMM' % (i*loop_size/1e6), end='\r')
    # fetch results from seattle city server
    try:
        results = client.get(data_mtd,
                         paidparkingarea='Belltown', 
                         select='occupancydatetime,paidoccupancy,parkingspacecount,parkingtimelimitcategory,sourceelementkey',
                         order='occupancydatetime',
                         limit=loop_size,
                         offset=loop_size * i)
    except:
        raise('Failed!')
    #convert to dataframe
    df = pd.DataFrame.from_records(results)

    # convert to appropriate data types
    df[num_cols] = df[num_cols].apply(pd.to_numeric)
    df[['occupancydatetime']] = df[['occupancydatetime']].apply(pd.to_datetime)    
    
    # get length of new dataframe
    df_len = len(df)
    #append dataframe to list of dataframe
    dfs.append(df)
    #update counter
    i += 1




In [51]:
results_df = pd.concat(dfs, ignore_index=True)

In [52]:
results_df.to_pickle("./30days.pkl")

In [53]:
results_df['sourceelementkey'].nunique()

251

In [55]:
print(len(dfs))

469
